In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
device = "cuda"
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR, CosineAnnealingWarmRestarts
import torchvision
from sklearn.model_selection import train_test_split
import torchvision.transforms as tt
import torchvision.models as models
from torchvision.datasets import CIFAR10, ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import random_split, DataLoader, Subset, TensorDataset, Dataset
from torch.nn.utils import vector_to_parameters, parameters_to_vector

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

import os
import time
import pickle

from PIL import Image

torch.manual_seed(0)
np.random.seed(0)

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class PathMNIST(Dataset):

    def __init__(self,
                 split='train',
                 transforms=None,
                 target_transform=None,
                 download=False):
        ''' dataset
        :param split: 'train', 'val' or 'test', select subset
        :param transform: data transformation
        :param target_transform: target transformation

        '''

        npz_file = np.load("/content/drive/MyDrive/PATH")

        self.split = split
        self.transforms = transforms
        self.transform_index = 0
        self.target_transform = target_transform
        #X = np.concatenate((npz_file['train_images'], npz_file['val_images']))
        #Y = np.concatenate((npz_file['train_labels'], npz_file['val_labels']))
        X_train = npz_file['train_images']
        Y_train = npz_file['train_labels']
        X_val = npz_file['val_images']
        Y_val = npz_file['val_labels']
        X_test = npz_file['test_images']
        Y_test = npz_file['test_labels']
        #print(X.shape)
        #X_train, X_val,Y_train,Y_val = train_test_split(X,Y,test_size = 0.2, random_state =0 )
        print(X_train.shape)
        print(X_val.shape)
        print(X_test.shape)
        if self.split == 'train':  # 89996 images
            self.img = X_train
            self.label = Y_train
        elif self.split == 'val':     #10004  images
            self.img = X_val
            self.label = Y_val
        elif self.split == 'test':      #7180 images
            self.img = X_test
            self.label = Y_test



    def __getitem__(self, index):
        img, target = self.img[index], self.label[index].astype(int)[0]
        img = Image.fromarray(np.uint8(img))

        if self.transforms is not None:
            img = self.transforms[self.transform_index](img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return self.img.shape[0]

In [4]:
light_transform = tt.Compose([
                              tt.ToTensor(),
                              tt.Lambda(lambda x : (x - x.mean())/(x.std()))
])

In [5]:
train_ds = PathMNIST(split='train', transforms= [light_transform])
val_ds = PathMNIST(split='val', transforms= [light_transform])
test_ds = PathMNIST(split='test', transforms= [light_transform])
train_dl = DataLoader(train_ds,512,shuffle = True,num_workers = 4, pin_memory = True)
test_dl = DataLoader(test_ds,512,True,num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_ds,512,True,num_workers = 4, pin_memory = True)

(89996, 28, 28, 3)
(10004, 28, 28, 3)
(7180, 28, 28, 3)
(89996, 28, 28, 3)
(10004, 28, 28, 3)
(7180, 28, 28, 3)
(89996, 28, 28, 3)
(10004, 28, 28, 3)
(7180, 28, 28, 3)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
BATCH_SIZE = 512
device = 'cuda'
NUM_CLASSES = 9

In [7]:
import torch
import torch.nn as nn

class CustomBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CustomBlock, self).__init__()
        # Define conv1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        # Define bn1 (GroupNorm with 2 groups for in_channels channels)
       # self.gn1 = nn.GroupNorm(2, in_channels, eps=1e-05, affine=True)
        # Define ReLU activation
        self.relu = nn.ReLU(inplace=True)
        # Define conv2
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        # Define bn2 (GroupNorm with 2 groups for out_channels channels)
        self.gn2 = nn.GroupNorm(2, out_channels, eps=1e-05, affine=True)
        # Define downsample
        self.downsample = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=(1, 1), stride=(2, 2), bias=False),
            nn.GroupNorm(32, out_channels, eps=1e-05, affine=True)
        )

    def forward(self, x):
        # Forward pass through conv1, bn1, and relu
        out = self.conv1(x)
       # out = self.gn1(out)
        out = self.relu(out)
        # Forward pass through conv2 and bn2
        out = self.conv2(out)
        out = self.gn2(out)

        # Downsampling path
        residual = self.downsample(x)

        # Add residual to the output
        out += residual

        out = self.conv2(out)
        out = self.gn2(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.gn2(out)


        return out

In [8]:
import torch
import torch.nn as nn

class ResNet_Part1(nn.Module):
    def __init__(self):
        super(ResNet_Part1, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.GroupNorm(32, 64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Layer 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False),
            nn.GroupNorm(2, 64, eps=1e-5, affine=True),
            nn.Conv2d(64, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False),
            nn.GroupNorm(2, 64, eps=1e-5, affine=True),

            nn.Conv2d(64, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False),
            nn.GroupNorm(2, 64, eps=1e-5, affine=True),
            nn.Conv2d(64, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False),
            nn.GroupNorm(2, 64, eps=1e-5, affine=True))


    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)

        return x



In [9]:
import torch
import torch.nn as nn

class ResNet_Part2(nn.Module):
    def __init__(self):
        super(ResNet_Part2, self).__init__()

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 9)

    def forward(self, x):

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x



In [10]:
class FullResNet(nn.Module):
    def __init__(self, model_part1, model_layer2, model_layer3, model_layer4, model_part2):
        super(FullResNet, self).__init__()
        # Store each part as a submodule
        self.model_part1 = model_part1
        self.model_layer2 = model_layer2
        self.model_layer3 = model_layer3
        self.model_layer4 = model_layer4
        self.model_part2 = model_part2

    def forward(self, x):
        # Pass the input through each part sequentially
        x = self.model_part1(x)    # Initial ResNet layers
        x = self.model_layer2(x)   # Custom block layer 2
        x = self.model_layer3(x)   # Custom block layer 3
        x = self.model_layer4(x)   # Custom block layer 4
        x = self.model_part2(x)    # Final ResNet layers
        return x

In [11]:
model_part1 = ResNet_Part1()
model_layer2 = CustomBlock(64, 128)
model_layer3 = CustomBlock(128, 256)
model_layer4 = CustomBlock(256, 512)
model_part2 = ResNet_Part2()

model = FullResNet(model_part1, model_layer2, model_layer3, model_layer4, model_part2)

In [12]:
model = model.to(device)

In [13]:
for name, module in model.named_modules():
    print(name, module)

 FullResNet(
  (model_part1): ResNet_Part1(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): GroupNorm(32, 64, eps=1e-05, affine=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): GroupNorm(2, 64, eps=1e-05, affine=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): GroupNorm(2, 64, eps=1e-05, affine=True)
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (5): GroupNorm(2, 64, eps=1e-05, affine=True)
      (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (7): GroupNorm(2, 64, eps=1e-05, affine=True)
    )
  )
  (model_layer2): CustomBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2

In [14]:
opt = SGD(model.parameters(),lr = 1e-3)
criterion = nn.CrossEntropyLoss()

In [15]:
BATCH_SIZE = 512
MAX_PHYSICAL_BATCH_SIZE = 128

In [18]:
import warnings
warnings.simplefilter("ignore")

MAX_GRAD_NORM = 4
EPSILON = 4
DELTA = 1e-5
EPOCHS = 50

LR = 1e-3

In [17]:
!pip install opacus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.9/239.9 kB 8.1 MB/s eta 0:00:00


In [19]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()

priv_model, priv_optimizer, priv_train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=opt,
    data_loader=train_dl,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)



In [20]:
print(f"Using sigma={priv_optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")

Using sigma=0.872802734375 and C=4


In [21]:
def accuracy(preds, labels):
    return (preds == labels).mean()

In [24]:
import numpy as np
from opacus.utils.batch_memory_manager import BatchMemoryManager

def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []

    with BatchMemoryManager(
        data_loader=train_loader,
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE,
        optimizer=optimizer
    ) as memory_safe_data_loader:
      for i, (images, target) in enumerate(memory_safe_data_loader):
          optimizer.zero_grad()
          images = images.to(device)
          target = target.to(device)

          # compute output
          output = model(images)
          loss = criterion(output, target)

          preds = np.argmax(output.detach().cpu().numpy(), axis=1)
          labels = target.detach().cpu().numpy()

          # measure accuracy and record loss
          acc = accuracy(preds, labels)

          losses.append(loss.item())
          top1_acc.append(acc)

          loss.backward()
          optimizer.step()
      print(
                  f"\tTrain Epoch: {epoch} \t"
                  f"Loss: {np.mean(losses):.6f} "
                  f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
              )

In [25]:
def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)

    print(
        f"\tTest set:"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)

In [26]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train(priv_model, priv_train_loader, priv_optimizer, epoch + 1, device)

Epoch:   0%|          | 0/50 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 2.103053 Acc@1: 21.253479 
	Train Epoch: 2 	Loss: 1.922962 Acc@1: 27.560778 
	Train Epoch: 3 	Loss: 1.826301 Acc@1: 33.025859 
	Train Epoch: 4 	Loss: 1.755302 Acc@1: 36.951538 
	Train Epoch: 5 	Loss: 1.691399 Acc@1: 39.698600 
	Train Epoch: 6 	Loss: 1.641146 Acc@1: 41.282930 
	Train Epoch: 7 	Loss: 1.599490 Acc@1: 42.410315 
	Train Epoch: 8 	Loss: 1.565543 Acc@1: 43.477714 
	Train Epoch: 9 	Loss: 1.533025 Acc@1: 44.560737 
	Train Epoch: 10 	Loss: 1.505092 Acc@1: 45.595451 
	Train Epoch: 11 	Loss: 1.498564 Acc@1: 45.630179 
	Train Epoch: 12 	Loss: 1.487338 Acc@1: 46.155791 
	Train Epoch: 13 	Loss: 1.479080 Acc@1: 46.677015 
	Train Epoch: 14 	Loss: 1.468608 Acc@1: 47.302790 
	Train Epoch: 15 	Loss: 1.467536 Acc@1: 47.532790 
	Train Epoch: 16 	Loss: 1.460467 Acc@1: 47.910629 
	Train Epoch: 17 	Loss: 1.451508 Acc@1: 48.519129 
	Train Epoch: 18 	Loss: 1.436533 Acc@1: 48.997014 
	Train Epoch: 19 	Loss: 1.437278 Acc@1: 49.125966 
	Train Epoch: 20 	Loss: 1.424461 Acc@1: 

In [27]:
torch.save(model.state_dict(),"only_opacus.pt")

In [29]:
test(priv_model,test_dl,"cuda")

	Test set:Loss: 1.214539 Acc: 61.736111 


0.617361111111111